In [16]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path
from glob import glob

import ijson
from tqdm import tqdm

## Import JSON 

In [53]:
zip_dir  = "../data/zip2"

files = glob(os.path.join(zip_dir, '*'))
for filename in files:
    if '.zip' in filename:
        pass
    elif '.json' in filename:
        pass
    else:
        newfile = filename+'.json'
        os.rename(filename, newfile)

jsons = glob(os.path.join(zip_dir, '*.json'))

In [118]:
# jsons = jsons[:2]
jsons

['../data/zip2/part-v003-o001-r-00000.json',
 '../data/zip2/part-v003-o001-r-00001.json',
 '../data/zip2/part-v003-o001-r-00002.json',
 '../data/zip2/part-v003-o001-r-00003.json',
 '../data/zip2/part-v003-o001-r-00004.json',
 '../data/zip2/part-v003-o001-r-00005.json',
 '../data/zip2/part-v003-o001-r-00006.json',
 '../data/zip2/part-v003-o001-r-00007.json',
 '../data/zip2/part-v003-o001-r-00008.json',
 '../data/zip2/part-v003-o001-r-00009.json']

In [54]:
# jsons = ['../data/part_v003_o001_r_00000.json', ..., '../data/part_v003_o001_r_00001.json']

dfrts = []
dfors = []
for i, json in enumerate(jsons):
    # Instance Preparation
    dates = []
    tweets = []
    user_ids = []
    rts = []
    
    start = time.time()
    # Load json file: date and tweet
    with open(json, 'r', encoding='utf8') as file:
        pet_parse = ijson.parse(file, multiple_values=True)
        for prefix, event, value  in pet_parse:
            # Date
            if prefix == 'created_at':
                dates.append(datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y'))
            # Tweet 
            if prefix == 'text':
                tweets.append(value.replace('\n', '').replace('\t', '').replace('\r', '').replace('　', '')) # Delte space and indet
            # User id 
            if prefix == 'user.id':
                user_ids.append(value)
            # RT Flag
            if len(dates)-1 == len(rts) and prefix == 'retweeted_status':
                rts.append(True)
            if len(dates)-2 == len(rts):
                rts.append(False)
                
        if len(dates) != len(rts):
                rts.append(False)
#
    print ("Loading json elapsed_time:{0}".format(time.time() - start) + "[sec]")
    
#     print(len(dates), len(tweets), len(user_ids), len(rts))
    
    data = np.vstack([user_ids, dates, tweets, rts]).T
    df = pd.DataFrame(data, columns=['user_id', 'date', 'tweet', 'RT_flag'])
    
    # Date
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['date'].dt.hour
    df['time'] = df['date'].dt.time
    df['date'] = df['date'].dt.date
    
    # divide df by Original tweet and RT 
    dfrt = df[df['RT_flag'] == True]
    dfor = df[df['RT_flag'] == False]
    
    # Save to csv file
    dfor.to_csv('../data/new_dfs2/dfor'+str(i)+'.csv')
    dfrt.to_csv('../data/new_dfs2/dfrt'+str(i)+'.csv')
    
#     # Append to dfs
#     dfrts.append(dfor)
#     dfors.append(dfor)

Loading json elapsed_time:707.7448446750641[sec]
Loading json elapsed_time:647.3039815425873[sec]
Loading json elapsed_time:527.2282273769379[sec]
Loading json elapsed_time:462.67126631736755[sec]
Loading json elapsed_time:460.99883365631104[sec]
Loading json elapsed_time:495.58344650268555[sec]
Loading json elapsed_time:511.23239040374756[sec]
Loading json elapsed_time:481.3941822052002[sec]
Loading json elapsed_time:468.47084856033325[sec]
Loading json elapsed_time:644.1469695568085[sec]


In [109]:
print(len(dates), len(tweets), len(user_ids), len(rts))

1175787 1175787 1175787 1175787


In [56]:
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 10000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 


In [55]:
del dates, tweets, rts, dfor, dfrt, df, user_ids

## Read json dfs from csv

In [17]:
zip_dir  = "../data/new_dfs2"
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    df = df.dropna(subset=['tweet'])
    dfors.append(df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
zip_dir  = "../data/dfs"
csvfiles = glob(os.path.join(zip_dir, 'dfrt*'))

dfrts = []
for csvfile in csvfiles:
    df1 = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    df1 = df1.dropna(subset=['tweet'])
    dfrts.append(df1)

## df info

In [39]:
d = dfors[2]
d.head()

,user_id,date,tweet,RT_flag,year,month,day,hour,time
0,1232197603896352769,2020-03-25,バズツイ様のおかげでSOUTUSや2getherからタイBLドラマにハマってくれた腐女子のみ...,False,2020.0,3,25.0,10.0,10:11:59
1,1931070408,2020-03-25,@aoihyouClio2RS憂鬱。,False,2020.0,3,25.0,10.0,10:11:59
2,1094052354112991232,2020-03-25,@I6TdGuyyxnDi6rSそだね！一番はそれだね💦ほんと毎月憂鬱だよね😫人生の半分はそ...,False,2020.0,3,25.0,10.0,10:11:58
3,1232871336956940289,2020-03-25,@schan4w1でもでも、私もですよ笑😅弱音は言わない人ですが、きっと初めての子の時、私の...,False,2020.0,3,25.0,10.0,10:11:58
4,799927833661345792,2020-03-25,@payopayooooいいなー自分流れ星流れてても気づけんから一生落ちてこん(^p^),False,2020.0,3,25.0,10.0,10:11:58


In [35]:
d.RT_flag.unique()

array(['False', nan, '25', False, '24', '23', '22'], dtype=object)

In [54]:
len(d[d['RT_flag']=='23'])

24

In [38]:
d.date.unique()

array(['2020-03-25', '2020', '2020-03-24', '2020-03-23', '2020-03-22'],
      dtype=object)

In [41]:
len(d.user_id)

903151

In [ ]:
# user_id, date, tweets がない

In [20]:
len(dfors)

10

In [ ]:
c = [len(df) for df in dfors]
print('The amount of tweets included RT:', sum(c))

In [22]:
c = [len(df) for df in dfors]
print('The amount of tweets included RT:', sum(c))

The amount of tweets included RT: 7968516


## Create csv by day

In [23]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_day(dfors, filetype, savedir, key_lang):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')
    
    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'
    
    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
            
    
    # tweets volume
    vol = []
    
    # Find tweets
    for dfor in tqdm(dfors):
        
        vol.append(len(dfor)) # tweets volume
        
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        # Add columns of keywords whose cell have 1 if this tweet includes a keyword 
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        # Make rows
        uni_dates = dfor_c['date'].tolist()
        uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
        rows = []
        for date in uni_dates:
            d = {}
            d['date'] = str(date)
            for col in filenames:
                _df = dfor_c.groupby('date').get_group(date)
                d[col] = _df[col].sum()
            rows.append(d)
        
        # Make cols
        cols = filenames.copy()
        cols.insert(0, 'date')
        
        # Make dfs with rows and cols
        dft = pd.DataFrame(columns=cols)
        for row in rows:
            dft = dft.append(row, ignore_index=True)
        
        dfs.append(dft)
    
    # Print volume of this tweets 
    print('Total tweets:', sum(vol))
    
    # Finally Connect dfs
    for i, df in enumerate(dfs):
        if i == 0:
            dfnew = df
        else:
            dfnew = pd.concat([dfnew, df], axis=0)
            
    # Groupby and sort by date
    dfnew = dfnew.groupby('date').sum()
    
    print('Original tweets volume:', sum(vol))
    
    # Save
    outname = filetype+'_original.csv'
    outdir = savedir
#     outdir = '../results2/rten/day'

    if not os.path.exists(outdir):
            os.makedirs(outdir)

    savename = os.path.join(outdir, outname)    

    dfnew.to_csv(savename)

    return dfnew


## Run day

In [25]:
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfors, filetype=filetype, savedir='../new_results/orjp/day', key_lang='jp')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [01:23<00:00,  8.38s/it]


Total tweets: 7968516


100%|██████████| 10/10 [02:01<00:00, 12.14s/it]


Original tweets volume: 7968516
Total tweets: 7968516
Original tweets volume: 7968516


100%|██████████| 10/10 [02:08<00:00, 12.86s/it]


...

Total tweets: 7968516
Original tweets volume: 7968516


  0%|          | 0/10 [00:00<?, ?it/s]

Total tweets: 7968516
Original tweets volume: 7968516


100%|██████████| 10/10 [02:01<00:00, 12.14s/it]


Total tweets: 7968516
Original tweets volume: 7968516


In [ ]:
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfrts, filetype=filetype, savedir='../new_results/rtjp/day', key_lang='jp')

## Create csv by hour 

In [26]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_hour(dfors, filetype, savedir, key_lang):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')

    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'

    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    
    # Keyword 
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        dfs.append(dfor_c)
        
    # Create csv file by day
    all_df = []
    nextdf, nextdf_date = '', ''
    
    # Save dir
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)

    for i, df in enumerate(dfs):
        unique_dates = df['date'].tolist()
        unique_dates = sorted(set(unique_dates), key=unique_dates.index) # date values
        df_date = df.groupby('date') # df grouped by date   
        
        dfhs = {}
        
        for date in unique_dates:
            df_oneday = df_date.get_group(date) # df of one day
            dfh = df_oneday.groupby('hour').sum()
            dfh = dfh.drop(['year', 'month'],axis=1)
            
            
            outname = filetype+str(date)+'.csv'
            savename = os.path.join(outdir, outname)
            
            if os.path.exists(savename):
                # File1
                existedfile = pd.read_csv(savename)
                
                # いったんcsvにする
                _saveonce = 'once.csv'
                dfh.to_csv(_saveonce)
                # File2 
                once = pd.read_csv(_saveonce)
                
                # Concat 
                df_oneday1 = pd.concat([existedfile, once], axis=0)

                # Finally save
                dfh = df_oneday1.groupby('hour').sum()
                
            # Save to csv
            dfh.to_csv(savename)
          

## Run Orjp, Oren, Rtjp, Rten

In [27]:
# Orjp
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfors, filetype=filetype, savedir='../new_results/orjp/hour', key_lang='jp')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [01:22<00:00,  8.26s/it]


KeyError: 'hour'

In [8]:
# Oren
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfors, filetype=filetype, savedir='../results2/oren/hour', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:52<00:00,  5.24s/it]


In [4]:
# Rtjp
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfrts, filetype=filetype, savedir='../results2/rtjp/hour', key_lang='jp')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [04:23<00:00, 26.37s/it]


In [5]:
# Rten
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfrts, filetype=filetype, savedir='../results2/rten/hour', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:44<00:00,  4.49s/it]


## Check 

In [77]:
dft = pd.read_csv('../results/rtjp/hour/T2020-02-28.csv')
dft.T1.sum()

7827

In [ ]:
	T1
21	2958
22	2637
23	2546
24	4883
25	3352
26	2553
27	3206
28	5117
29	7827
30	2925

## コロナ含有率

In [15]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_corona(dfors, filetype="T"):
    # Prepration of keywords
    ## コロナ専用
    keywords = ['コロナ']
    
    c = [] # count volume
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        c.append(len(dfor_c))
    
    print(sum(c), 'tweets conatin "コロナ"')

# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
findtwt_corona(dfors, filetype="T")
          

100%|██████████| 10/10 [00:13<00:00,  1.32s/it]

965457 tweets conatin "コロナ"


## Corona by day

In [3]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwtCrn_day(dfors, savename, savedir):
    
    dfs = []

    # Keyowrds
    keywords = ['コロナ', 'Corona', 'corona']
    filenames = ['コロナ', 'Corona', 'corona']

    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet include keywords
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        # Make rows
        uni_dates = dfor_c['date'].tolist()
        uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
        rows = []
        for date in uni_dates:
            d = {}
            d['date'] = str(date)
            for col in filenames:
                _df = dfor_c.groupby('date').get_group(date)
                d[col] = _df[col].sum()
            rows.append(d)
        
        # Make cols
        cols = filenames.copy()
        cols.insert(0, 'date')
        
        # Make dfs with rows and cols
        dft = pd.DataFrame(columns=cols)
        for row in rows:
            dft = dft.append(row, ignore_index=True)
        
        dfs.append(dft)

    # Finally Connect dfs
    for i, df in enumerate(dfs):
        if i == 0:
            dfnew = df
        else:
            dfnew = pd.concat([dfnew, df], axis=0)
            
    # Groupby and sort by date
    dfnew = dfnew.groupby('date').sum()
    
    # Save
    outname = savename+'.csv'
    outdir = savedir

    if not os.path.exists(outdir):
            os.makedirs(outdir)

    savename = os.path.join(outdir, outname)    
    dfnew.to_csv(savename)

    return dfnew

In [4]:
dfnew = findtwtCrn_day(dfors, savename='corona', savedir='../results/Corona/or/')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:45<00:00,  4.52s/it]


In [21]:
dfnew = findtwtCrn_day(dfrts, savename='corona', savedir='../results/Corona/rt/')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:35<00:00,  3.56s/it]


## Corona by hour

In [9]:
def findtwtCrn_hour(dfors, savename, savedir):
    # Prepartion 
    results = []
    dfs = []
    
    # Save dir
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)
    
    # Keyowrds
    keywords = ['コロナ', 'Corona', 'corona']
    filenames = ['コロナ', 'Corona', 'corona']
    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        dfs.append(dfor_c)
        
    # Create csv file by day
    all_df = []
    nextdf, nextdf_date = '', ''
    
    # Save dir
    outdir = savedir
    filetype = savename
    if not os.path.exists(outdir):
            os.makedirs(outdir)

    for i, df in enumerate(dfs):
        unique_dates = df['date'].tolist()
        unique_dates = sorted(set(unique_dates), key=unique_dates.index) # date values
        df_date = df.groupby('date') # df grouped by date   
        
        dfhs = {}
        
        for date in unique_dates:
            df_oneday = df_date.get_group(date) # df of one day
            dfh = df_oneday.groupby('hour').sum()
            dfh = dfh.drop(['year', 'month'],axis=1)
            
            
            outname = filetype+str(date)+'.csv'
            savename = os.path.join(outdir, outname)
            
            if os.path.exists(savename):
                # File1
                existedfile = pd.read_csv(savename)
                
                # いったんcsvにする
                _saveonce = 'once.csv'
                dfh.to_csv(_saveonce)
                # File2 
                once = pd.read_csv(_saveonce)
                
                # Concat 
                df_oneday1 = pd.concat([existedfile, once], axis=0)

                # Finally save
                dfh = df_oneday1.groupby('hour').sum()
                
            # Save to csv
            dfh.to_csv(savename)
          

In [10]:
findtwtCrn_hour(dfors, savename='corona', savedir='../results/Corona/or')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:28<00:00,  2.87s/it]


In [22]:
findtwtCrn_hour(dfrts, savename='corona', savedir='../results/Corona/rt/')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:39<00:00,  3.94s/it]


In [19]:
dft = pd.read_csv('../results/Corona/or/corona2020-02-27.csv')
dft['コロナ'].sum()

18993

In [ ]:
	コロナ
26	13235
27	16052
28	18993
29	18487
30	11724